In [ ]:
#!pip install gymnasium[box2d] swig box2d-py gymnasium stable_baselines3[extra] box2d box2d-kengz ipywidgets ffmpeg

In [ ]:
!apt install swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,459 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Create environment
env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Instantiate the agent
model = PPO(
    'MlpPolicy',
    env,
    verbose=1,
    ##### HYPERPARAMETERS HERE!!!!
    learning_rate=0.001,
    batch_size=32,
    )

# Train the agent and display a progress bar
model.learn(total_timesteps=int(10000), progress_bar=True)

# Save the agent
model.save("ppo_lunar")
#del model  # delete trained model to demonstrate loading

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 97.4     |
|    ep_rew_mean     | -221     |
| time/              |          |
|    fps             | 771      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 94.9        |
|    ep_rew_mean          | -186        |
| time/                   |             |
|    fps                  | 448         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011201427 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.38       |
|    explained_variance   | 0.00817     |
|    learning_rate        | 0.

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import imageio
import numpy as np

# Load the trained agent
env = gym.make("LunarLander-v2", render_mode="rgb_array")
model = PPO.load("ppo_lunar", env=env)

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print('Mean reward:', mean_reward, 'Std. reward:', std_reward)

# Test the trained agent
images = []
j = 0
vec_env = model.get_env()
obs = vec_env.reset()
img = model.env.render(mode="rgb_array")
for i in range(2000):
    images.append(img)
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    #vec_env.render()
    if dones:
        j = j + 1
        print('Episode:', j, 'Rewards:', rewards, 'Info:', info, 'Dones:', dones)
        imageio.mimsave(f"ppo_lander_{j}.gif", [np.array(img) for k, img in enumerate(images) if k%2 == 0], fps=29)
        images = []
    img = model.env.render(mode="rgb_array")
vec_env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Mean reward: -509.8155554000001 Std. reward: 137.21080950391766
Episode: 1 Rewards: [-100.] Info: [{'episode': {'r': -451.617524, 'l': 166, 't': 4.625208}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 1.010795  ,  3.1059046 ,  1.1984864 , -0.04458454, -0.5789374 ,
       -0.11271373,  0.        ,  0.        ], dtype=float32)}] Dones: [ True]


/usr/local/lib/python3.10/dist-packages/imageio/plugins/pillow.py:390: DeprecationWarning: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == `duration=20` (1000 * 1/50).
  warnings.warn(


Episode: 2 Rewards: [-100.] Info: [{'episode': {'r': -971.379684, 'l': 340, 't': 8.449809}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 1.0082937 ,  7.5434785 ,  1.6206703 ,  1.0692391 , -0.55168426,
       -0.10118358,  0.        ,  0.        ], dtype=float32)}] Dones: [ True]
Episode: 3 Rewards: [-100.] Info: [{'episode': {'r': -373.46032, 'l': 289, 't': 15.352019}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 1.0006866 ,  2.125324  ,  1.0866307 , -0.16745566, -0.77195156,
       -0.6855892 ,  0.        ,  0.        ], dtype=float32)}] Dones: [ True]
Episode: 4 Rewards: [-100.] Info: [{'episode': {'r': -499.990495, 'l': 183, 't': 19.84825}, 'TimeLimit.truncated': False, 'terminal_observation': array([ 1.0123322 ,  3.5343037 ,  1.3150403 ,  0.24421243, -0.45588028,
       -0.08848883,  0.        ,  0.        ], dtype=float32)}] Dones: [ True]
Episode: 5 Rewards: [-100.] Info: [{'episode': {'r': -306.951208, 'l': 123, 't': 22.259099}, 'TimeLimit.trunc